In [ ]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
turbmethod1 = 'SMCLT-H15'
turbmethod2 = 'SMCLT-KC04'
turbmethod3 = 'KPPLT-LF17'

datapath = os.path.join(os.path.pardir, 'tests', '{:s}'.format(casename))
filepath = os.path.join(datapath, 'averages.jld2')
data_pfl = OceananigansDataProfile(filepath=filepath)
les_ds = data_pfl.dataset

gotm_dir   = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}'.format(casename))
gotm_sim1  = Simulation(path=os.path.join(gotm_dir, turbmethod1+'-Rlx'))
gotm_sim2  = Simulation(path=os.path.join(gotm_dir, turbmethod2+'-Rlx'))
gotm_sim3  = Simulation(path=os.path.join(gotm_dir, turbmethod3+'-Rlx'))
gotm_ds1   = gotm_sim1.load_data()
gotm_ds2   = gotm_sim2.load_data()
gotm_ds3   = gotm_sim3.load_data()

In [ ]:
figpath  = os.path.join(os.path.pardir, 'gotm', 'figure', '{:s}-cmpRlex'.format(casename))
os.makedirs(figpath, exist_ok=True)

In [ ]:
g = 9.81
H = 30
u10 = 8
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)

In [ ]:
# one inertial period (17 hours)
startdate1 = '2000-01-04T00:00:00'
enddate1 = '2000-01-04T17:00:00'
tslice1 = slice(startdate1, enddate1)
startdate2 = '2000-01-10T00:00:00'
enddate2 = '2000-01-10T17:00:00'
tslice2 = slice(startdate2, enddate2)
startdate3 = '2000-01-12T00:00:00'
enddate3 = '2000-01-12T17:00:00'
tslice3 = slice(startdate3, enddate3)

In [ ]:
def get_flux(lam, num, gam):
    tmp = xr.zeros_like(num)
    nt = lam.shape[1]
    for i in np.arange(nt):
        tmp.data[1:-1,i] = (lam.data[:-1,i]-lam.data[1:,i])/(lam.z.data[:-1]-lam.z.data[1:])
    flux = - num * tmp + gam
    return flux


In [ ]:
wu_les = (les_ds.data_vars['wu']+les_ds.data_vars['wusb'])/ustar**2
ds_list = [gotm_ds1, gotm_ds2, gotm_ds3]
wu_gotm = [(get_flux(ds.data_vars['u'].squeeze(), ds.data_vars['num'].squeeze(), ds.data_vars['gamu'].squeeze())
      -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze())/ustar**2 for ds in ds_list]

In [ ]:
fig, axarr = plt.subplots(1, 2, sharey='row')
fig.set_size_inches([6,3])

wu_les.sel(time=tslice1).mean(dim='time').plot(ax=axarr[0], y=wu_les.dims[0], color='k', linestyle='-', label='LES')
wu_gotm[0].sel(time=tslice1).mean(dim='time').plot(ax=axarr[0], y=wu_gotm[0].dims[0], color='tab:blue', linestyle='-', label=turbmethod1)
wu_gotm[1].sel(time=tslice1).mean(dim='time').plot(ax=axarr[0], y=wu_gotm[1].dims[0], color='tab:red', linestyle='-', label=turbmethod2)
wu_gotm[2].sel(time=tslice1).mean(dim='time').plot(ax=axarr[0], y=wu_gotm[2].dims[0], color='tab:purple', linestyle='-', label=turbmethod3)

wu_les.sel(time=tslice2).mean(dim='time').plot(ax=axarr[1], y=wu_les.dims[0], color='k', linestyle='-', label='LES')
wu_gotm[0].sel(time=tslice2).mean(dim='time').plot(ax=axarr[1], y=wu_gotm[0].dims[0], color='tab:blue', linestyle='-', label=turbmethod1)
wu_gotm[1].sel(time=tslice3).mean(dim='time').plot(ax=axarr[1], y=wu_gotm[1].dims[0], color='tab:red', linestyle='-', label=turbmethod2)
wu_gotm[2].sel(time=tslice2).mean(dim='time').plot(ax=axarr[1], y=wu_gotm[2].dims[0], color='tab:purple', linestyle='-', label=turbmethod3)

ylabel = ['Depth [m]', '']
title = ['Time averaged T1', 'Time averaged T2']
for j in np.arange(2):
    axarr[j].set_ylim([-H, 0])
    axarr[j].set_xlim([-2.4, 0.6])
    axarr[j].set_xlabel('$\overline{w^\prime u^\prime}/u_*^2$')
    axarr[j].set_ylabel(ylabel[j])
    axarr[j].set_title(title[j])
    axarr[j].axvline(x=0, linestyle='-', color='gray', linewidth=0.5)
axarr[0].legend()
plt.tight_layout()

figname2 = os.path.join(figpath, 'cmp_profile_wu')
fig.savefig(figname2, dpi = 300, facecolor='w')